In [1]:
# Advent of Code 2023
# Day 17 Problem 1
import re
from collections import Counter
import functools

with open("aoc_17_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}:\t{line}")

0:	2413432311323
1:	3215453535623
2:	3255245654254
3:	3446585845452
4:	4546657867536
5:	1438598798454
6:	4457876987766
7:	3637877979653
8:	4654967986887
9:	4564679986453
10:	1224686865563
11:	2546548887735
12:	4322674655533


In [15]:
import heapq
from collections import defaultdict

def parse(inp):
    return [[int(x) for x in row] for row in inp]

# trying manhattan distance
def heur(x, y, M):
    H = len(M) - 1
    W = len(M[0]) - 1
    return abs(H - x) + abs(W - y)

def max3AStar(start, goal, M):
    # A* selects the path that minimizes f(n) = g(n) + h(n)
    # n is next node
    # g(n) is the cost of the path from the start (current heat)
    # h(n) is a heuristic that estimates the cost of the cheapest path from n to goal
    # === 
    # N 0, E 1, S 2, W 3 
    # x y dir heat 
    frontier = []
    gScore = defaultdict(lambda: defaultdict(lambda:999_999_999))
    gScore[start] = {0:0, 1:0, 2:0, 3:0} 
    fScore = defaultdict(lambda: defaultdict(lambda:999_999_999))
    startHeur = heur(*start,M)
    fScore[start] = {0:startHeur, 1:startHeur, 2:startHeur, 3:startHeur}
    for x in [0,1,2,3]:
        heapq.heappush(frontier, (x+1, (*start, x, 0)))
    H = len(M)
    W = len(M[0])
    dirs = {
        (0,1): 1,
        (0,-1): 3,
        (1,0): 2,
        (-1,0): 0
    }

    while frontier:
        _, (x,y,dir,heat) = heapq.heappop(frontier)
        if (x,y) == goal:
            return heat

        # ignore the entire direction we came, forward and back
        if dir % 2 == 1:    options = {(1,0), (-1, 0)} 
        else:               options = {(0,1), (0,-1)}

        for (dx,dy) in options:
            # consider all 3 steps at once
            buildingHeat = 0
            dd = dirs[(dx,dy)]
            for i in [1,2,3]:
                xx = x + dx * i
                yy = y + dy * i
                if xx < 0 or yy < 0 or xx >= H or yy >= W:
                    continue

                buildingHeat += M[xx][yy]
                tentativeG = heat + buildingHeat
                
                # if this is a better path than what we have...
                if tentativeG < gScore[(xx,yy)][dd]:
                    gScore[(xx,yy)][dd] = tentativeG
                    currH = heur(xx,yy,M)
                    fScore[(xx,yy)][dd] = tentativeG + currH
                    nextCand =  (xx,yy,dd,tentativeG)
                    priority = tentativeG + currH
                    heapq.heappush(frontier, (priority, nextCand))

def p1(inp):
    heatLoss = parse(inp)
    H = len(inp)
    W = len(inp[0])
    totalHeat = max3AStar((0,0), (H-1, W-1), heatLoss)
    print(f"Total heat is {totalHeat}")
            

In [16]:
# Expected output: 102
p1(TEST)

Total heat is 102


In [17]:
p1(A)

Total heat is 886


In [5]:

def sweetSpotAStart(start, goal, M, mindist, maxdist):
    # A* selects the path that minimizes f(n) = g(n) + h(n)
    # n is next node
    # g(n) is the cost of the path from the start (current heat)
    # h(n) is a heuristic that estimates the cost of the cheapest path from n to goal
    # === 
    # N 0, E 1, S 2, W 3 
    # x y dir heat 
    frontier = []
    gScore = defaultdict(lambda: defaultdict(lambda:999_999_999))
    gScore[start] = {0:0, 1:0, 2:0, 3:0} 
    fScore = defaultdict(lambda: defaultdict(lambda:999_999_999))
    startHeur = heur(*start,M)
    fScore[start] = {0:startHeur, 1:startHeur, 2:startHeur, 3:startHeur}
    for x in [0,1,2,3]:
        heapq.heappush(frontier, (x+1, (*start, x, 0)))
    H = len(M)
    W = len(M[0])

    while frontier:
        _, (x,y,dir,heat) = heapq.heappop(frontier)
        if (x,y) == goal:
            return heat

        for dd,(dx,dy) in enumerate([(-1,0), (0,1), (1,0), (0,-1)]):
            # ignore the entire direction, forward and back, we came
            if dd == (dir + 2) % 4 or dd == dir:
                continue

            # consider all (mindist..maxdist) steps at once
            for i in range(mindist, maxdist+1):
                xx = x + dx * i
                yy = y + dy * i
                if xx < 0 or yy < 0 or xx >= H or yy >= W:
                    continue

                if dd == 1:     delta = sum(M[x][y+1:yy+1])
                elif dd == 3:   delta = sum(M[x][yy:y])
                elif dd == 0:   delta = sum([M[xxx][y] for xxx in range(xx,x)])
                else:           delta = sum([M[xxx][y] for xxx in range(x+1,xx+1)])
                tentG = heat + delta

                # if this is a better path than what we have...
                if tentG < gScore[(xx,yy)][dd]:
                    gScore[(xx,yy)][dd] = tentG
                    candH = heur(xx,yy,M)
                    fScore[(xx,yy)][dd] = tentG + candH
                    nextCand = (xx,yy,dd,tentG)
                    heapq.heappush(frontier, (tentG + candH, nextCand))

def p2(inp):
    heatLoss = parse(inp)
    H = len(inp)
    W = len(inp[0])
    totalHeat= sweetSpotAStart((0,0), (H-1, W-1), heatLoss, 4, 10)
    print(f"Total heat is {totalHeat}")

In [6]:
# expected output: 94
p2(TEST)

Total heat is 94


In [7]:
TESTB = """111111111111
999999999991
999999999991
999999999991
999999999991""".split("\n")

# expected output: 71
p2(TESTB)

Total heat is 71


In [8]:
p2(A)
# 900 is too low but the right answer for someone else :O

Total heat is 1055


In [ ]:
# I did A* here but really Dijkstra's would have been easier and faster and gotten the same result
# since my heuristic function (manhattan dist) is not actually doing much for me here and is 
# probably a detriment in p2.